In [1]:
import os
import pandas as pd
import openpyxl

# Path to the Excel file
old_listings_path = '../data/landing/old_listing/Moving annual rent by suburb - March quarter 2023.xlsx'

# Read the Excel file
df = pd.read_excel(old_listings_path)

# Display the data
print(df.head())

  Moving annual rent by suburb                             Unnamed: 1  \
0               1 bedroom flat                                    NaN   
1                          NaN                                    NaN   
2              Inner Melbourne  Albert Park-Middle Park-West St Kilda   
3                          NaN                               Armadale   
4                          NaN                          Carlton North   

  Lease commenced in year ending  Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6  \
0                        Mar 2000        NaN   Jun 2000        NaN   Sep 2000   
1                           Count     Median      Count     Median      Count   
2                             352        165        347        165        378   
3                             210        150        212        150        213   
4                              87        150         78        155         74   

  Unnamed: 7 Unnamed: 8 Unnamed: 9  ... Unnamed: 178 Unnamed: 179  \
0    

1 bedroom flat
2 bedroom flat
3 bedroom flat

2 bedroom house
3 bedroom house
4 bedroom house

all properties

**1 bedroom flat**

In [2]:
import os
import pandas as pd
import openpyxl

# Path to the Excel file
old_listings_path = '../data/landing/old_listing/Moving annual rent by suburb - March quarter 2023.xlsx'

# Read the "1 bedroom flat" sheet
sheet_name = '1 bedroom flat'
df = pd.read_excel(old_listings_path, sheet_name=sheet_name)

# Strip leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Print the sheet name
print(f"\nData from sheet: {sheet_name}")

# Display the data
print(df)


Data from sheet: 1 bedroom flat
    Moving annual rent by suburb                             Unnamed: 1  \
0                 1 bedroom flat                                    NaN   
1                            NaN                                    NaN   
2                Inner Melbourne  Albert Park-Middle Park-West St Kilda   
3                            NaN                               Armadale   
4                            NaN                          Carlton North   
..                           ...                                    ...   
156                          NaN                            Wanagaratta   
157                          NaN                               Warragul   
158                          NaN                            Warrnambool   
159                          NaN                                Wodonga   
160                          NaN                            Group Total   

    Lease commenced in year ending Unnamed: 3 Unnamed: 4 Unnamed: 

In [9]:
import os
import pandas as pd
import openpyxl
import folium
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster

# Path to the Excel file
old_listings_path = '../data/landing/old_listing/Moving annual rent by suburb - March quarter 2023.xlsx'

# Read the "1 bedroom flat" sheet, skipping the first row
sheet_name = '1 bedroom flat'
df = pd.read_excel(old_listings_path, sheet_name=sheet_name, skiprows=1)

# Strip leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Filter data for "March 2023"
march_2023_data = df.iloc[:, [1, -2]].dropna()  # Assuming the last two columns are 'Count' and 'Median' for 'Mar 2023'

# Extract suburb names and counts
suburbs = march_2023_data.iloc[:, 0].dropna().unique()
counts = march_2023_data.iloc[:, 1].dropna().values

# Initialize geolocator
geolocator = Nominatim(user_agent="geoapiExercises")

# Create a map centered around Melbourne
melbourne_map = folium.Map(location=[-37.8136, 144.9631], zoom_start=10)

# Add markers to the map with different colors for each suburb
marker_cluster = MarkerCluster().add_to(melbourne_map)
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

for i, suburb in enumerate(suburbs):
    location = geolocator.geocode(f"{suburb}, Melbourne, Australia")
    if location:
        folium.Marker(
            location=[location.latitude, location.longitude],
            popup=f"{suburb}: {counts[i]}",
            icon=folium.Icon(color=colors[i % len(colors)])
        ).add_to(marker_cluster)

# Add a compact legend to the map
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 150px; height: auto; 
            border:2px solid grey; z-index:9999; font-size:12px;
            background-color:white; opacity: 0.8; padding: 10px;">
    <b>Legend</b><br>
'''

for i, suburb in enumerate(suburbs):
    legend_html += f'<i class="fa fa-map-marker fa-1x" style="color:{colors[i % len(colors)]}"></i>&nbsp;{suburb}<br>'

legend_html += '</div>'

melbourne_map.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file
melbourne_map.save('melbourne_1_bedroom_flats_march_2023.html')

# Display the map
melbourne_map

In [12]:
import os
import pandas as pd
import openpyxl
import folium
from geopy.geocoders import Photon
from folium.plugins import MarkerCluster
import time

# Path to the Excel file
old_listings_path = '../data/landing/old_listing/Moving annual rent by suburb - March quarter 2023.xlsx'

# Read the "1 bedroom flat" sheet, skipping the first row
sheet_name = '1 bedroom flat'
df = pd.read_excel(old_listings_path, sheet_name=sheet_name, skiprows=1)

# Strip leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Filter data for "March 2023"
march_2023_data = df.iloc[:, [1, -2]].dropna()  # Assuming the last two columns are 'Count' and 'Median' for 'Mar 2023'

# Extract suburb names and counts
suburbs = march_2023_data.iloc[:, 0].dropna().unique()
counts = march_2023_data.iloc[:, 1].dropna().values

# Initialize geolocator with increased timeout
geolocator = Photon(user_agent="geoapiExercises", timeout=10)

# Create a map centered around Melbourne
melbourne_map = folium.Map(location=[-37.8136, 144.9631], zoom_start=10)

# Add markers to the map with different colors for each suburb
marker_cluster = MarkerCluster().add_to(melbourne_map)
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

# Log missing locations
missing_locations = []

for i, suburb in enumerate(suburbs):
    try:
        location = geolocator.geocode(f"{suburb}, Melbourne, Australia")
        if location:
            folium.Marker(
                location=[location.latitude, location.longitude],
                popup=f"{suburb}: {counts[i]}",
                icon=folium.Icon(color=colors[i % len(colors)])
            ).add_to(marker_cluster)
        else:
            missing_locations.append(suburb)
    except Exception as e:
        print(f"Error geocoding {suburb}: {e}")
        missing_locations.append(suburb)
    time.sleep(1)  # Add a delay between requests

# Print missing locations
if missing_locations:
    print("The following suburbs could not be geocoded:")
    for suburb in missing_locations:
        print(suburb)

# Add a compact legend to the map
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 150px; height: auto; 
            border:2px solid grey; z-index:9999; font-size:12px;
            background-color:white; opacity: 0.8; padding: 10px;">
    <b>Legend</b><br>
'''

for i, suburb in enumerate(suburbs):
    legend_html += f'<i class="fa fa-map-marker fa-1x" style="color:{colors[i % len(colors)]}"></i>&nbsp;{suburb}<br>'

legend_html += '</div>'

melbourne_map.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file
melbourne_map.save('melbourne_1_bedroom_flats_march_2023.html')

# Display the map
melbourne_map

The following suburbs could not be geocoded:
Bulleen-Templestowe-Doncaster
Doncaster East-Donvale
Vermont-Forest Hill-Burwood East
Aspendale-Chelsea-Carrum
Bundoora-Greensborough-Hurstbridge
Eltham-Research-Montmorency
Dromana-Portsea
Sebastopol-Delacombe
Wendouree-Alfredton


**2 bedroom flat**

In [4]:
import os
import pandas as pd
import openpyxl

# Path to the Excel file
old_listings_path = '../data/landing/old_listing/Moving annual rent by suburb - March quarter 2023.xlsx'

# Read the "2 bedroom flat" sheet
sheet_name = '2 bedroom flat'
df = pd.read_excel(old_listings_path, sheet_name=sheet_name)

# Strip leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Print the sheet name
print(f"\nData from sheet: {sheet_name}")

# Display the data
print(df)


Data from sheet: 2 bedroom flat
    Moving annual rent by suburb                             Unnamed: 1  \
0                 2 bedroom flat                                    NaN   
1                            NaN                                    NaN   
2                Inner Melbourne  Albert Park-Middle Park-West St Kilda   
3                            NaN                               Armadale   
4                            NaN                          Carlton North   
5                            NaN                      Carlton-Parkville   
6                            NaN                        CBD-St Kilda Rd   
7                            NaN                 Collingwood-Abbotsford   
8                            NaN                              Docklands   
9                            NaN                         East Melbourne   
10                           NaN                          East St Kilda   
11                           NaN                                 El

3 bedroom flat

In [5]:
import os
import pandas as pd
import openpyxl

# Path to the Excel file
old_listings_path = '../data/landing/old_listing/Moving annual rent by suburb - March quarter 2023.xlsx'

# Read the "3 bedroom flat" sheet
sheet_name = '3 bedroom flat'
df = pd.read_excel(old_listings_path, sheet_name=sheet_name)

# Strip leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Print the sheet name
print(f"\nData from sheet: {sheet_name}")

# Display the data
print(df)


Data from sheet: 3 bedroom flat
    Moving annual rent by suburb                             Unnamed: 1  \
0                 3 bedroom flat                                    NaN   
1                            NaN                                    NaN   
2                Inner Melbourne  Albert Park-Middle Park-West St Kilda   
3                            NaN                               Armadale   
4                            NaN                          Carlton North   
5                            NaN                      Carlton-Parkville   
6                            NaN                        CBD-St Kilda Rd   
7                            NaN                 Collingwood-Abbotsford   
8                            NaN                              Docklands   
9                            NaN                         East Melbourne   
10                           NaN                          East St Kilda   
11                           NaN                                 El

2 bedroom house

In [6]:
import os
import pandas as pd
import openpyxl

# Path to the Excel file
old_listings_path = '../data/landing/old_listing/Moving annual rent by suburb - March quarter 2023.xlsx'

# Read the "2 bedroom house" sheet
sheet_name = '2 bedroom house'
df = pd.read_excel(old_listings_path, sheet_name=sheet_name)

# Strip leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Print the sheet name
print(f"\nData from sheet: {sheet_name}")

# Display the data
print(df)


Data from sheet: 2 bedroom house
    Moving annual rent by suburb                             Unnamed: 1  \
0                2 bedroom house                                    NaN   
1                            NaN                                    NaN   
2                Inner Melbourne  Albert Park-Middle Park-West St Kilda   
3                            NaN                               Armadale   
4                            NaN                          Carlton North   
5                            NaN                      Carlton-Parkville   
6                            NaN                        CBD-St Kilda Rd   
7                            NaN                 Collingwood-Abbotsford   
8                            NaN                              Docklands   
9                            NaN                         East Melbourne   
10                           NaN                          East St Kilda   
11                           NaN                                 E

3 bedroom house

In [7]:
import os
import pandas as pd
import openpyxl

# Path to the Excel file
old_listings_path = '../data/landing/old_listing/Moving annual rent by suburb - March quarter 2023.xlsx'

# Read the "3 bedroom house" sheet
sheet_name = '3 bedroom house'
df = pd.read_excel(old_listings_path, sheet_name=sheet_name)

# Strip leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Print the sheet name
print(f"\nData from sheet: {sheet_name}")

# Display the data
print(df)


Data from sheet: 3 bedroom house
    Moving annual rent by suburb                             Unnamed: 1  \
0                3 bedroom house                                    NaN   
1                            NaN                                    NaN   
2                Inner Melbourne  Albert Park-Middle Park-West St Kilda   
3                            NaN                               Armadale   
4                            NaN                          Carlton North   
5                            NaN                      Carlton-Parkville   
6                            NaN                        CBD-St Kilda Rd   
7                            NaN                 Collingwood-Abbotsford   
8                            NaN                              Docklands   
9                            NaN                         East Melbourne   
10                           NaN                          East St Kilda   
11                           NaN                                 E

4 bedroom house

In [8]:
import os
import pandas as pd
import openpyxl

# Path to the Excel file
old_listings_path = '../data/landing/old_listing/Moving annual rent by suburb - March quarter 2023.xlsx'

# Read the "4 bedroom house" sheet
sheet_name = '4 bedroom house'
df = pd.read_excel(old_listings_path, sheet_name=sheet_name)

# Strip leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Print the sheet name
print(f"\nData from sheet: {sheet_name}")

# Display the data
print(df)


Data from sheet: 4 bedroom house
    Moving annual rent by suburb                             Unnamed: 1  \
0                4 bedroom house                                    NaN   
1                            NaN                                    NaN   
2                Inner Melbourne  Albert Park-Middle Park-West St Kilda   
3                            NaN                               Armadale   
4                            NaN                          Carlton North   
5                            NaN                      Carlton-Parkville   
6                            NaN                        CBD-St Kilda Rd   
7                            NaN                 Collingwood-Abbotsford   
8                            NaN                              Docklands   
9                            NaN                         East Melbourne   
10                           NaN                          East St Kilda   
11                           NaN                                 E

all properties

In [9]:
import os
import pandas as pd
import openpyxl

# Path to the Excel file
old_listings_path = '../data/landing/old_listing/Moving annual rent by suburb - March quarter 2023.xlsx'

# Read the "All properties" sheet
sheet_name = 'All properties'
df = pd.read_excel(old_listings_path, sheet_name=sheet_name)

# Strip leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Print the sheet name
print(f"\nData from sheet: {sheet_name}")

# Display the data
print(df)


Data from sheet: All properties
    Moving annual rent by suburb                             Unnamed: 1  \
0                 All properties                                    NaN   
1                            NaN                                    NaN   
2                Inner Melbourne  Albert Park-Middle Park-West St Kilda   
3                            NaN                               Armadale   
4                            NaN                          Carlton North   
5                            NaN                      Carlton-Parkville   
6                            NaN                        CBD-St Kilda Rd   
7                            NaN                 Collingwood-Abbotsford   
8                            NaN                              Docklands   
9                            NaN                         East Melbourne   
10                           NaN                          East St Kilda   
11                           NaN                                 El

In [10]:
import os
import pandas as pd
import openpyxl

# Path to the Excel file
old_listings_path = '../data/landing/old_listing/Moving annual rent by suburb - March quarter 2023.xlsx'

# List of sheet names to read
sheets = [
    '1 bedroom flat', '2 bedroom flat', '3 bedroom flat',
    '2 bedroom house', '3 bedroom house', '4 bedroom house',
    'All properties'
]

# Read and display the number of rows for each sheet
for sheet in sheets:
    df = pd.read_excel(old_listings_path, sheet_name=sheet)
    
    # Strip leading/trailing spaces from column names
    df.columns = df.columns.str.strip()
    
    # Print the sheet name and number of rows
    print(f"\nSheet: {sheet}")
    print(f"Number of rows: {df.shape[0]}")


Sheet: 1 bedroom flat
Number of rows: 161

Sheet: 2 bedroom flat
Number of rows: 161

Sheet: 3 bedroom flat
Number of rows: 161

Sheet: 2 bedroom house
Number of rows: 161

Sheet: 3 bedroom house
Number of rows: 161

Sheet: 4 bedroom house
Number of rows: 161

Sheet: All properties
Number of rows: 161


In [11]:
import os
import pandas as pd
import openpyxl

# Path to the Excel file
old_listings_path = '../data/landing/old_listing/Moving annual rent by suburb - March quarter 2023.xlsx'

# Read the "All properties" sheet
sheet_name = 'All properties'
df = pd.read_excel(old_listings_path, sheet_name=sheet_name)

# Strip leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Print the sheet name
print(f"\nData from sheet: {sheet_name}")

# Display the first two columns
print(df.iloc[:, :2])


Data from sheet: All properties
    Moving annual rent by suburb                             Unnamed: 1
0                 All properties                                    NaN
1                            NaN                                    NaN
2                Inner Melbourne  Albert Park-Middle Park-West St Kilda
3                            NaN                               Armadale
4                            NaN                          Carlton North
5                            NaN                      Carlton-Parkville
6                            NaN                        CBD-St Kilda Rd
7                            NaN                 Collingwood-Abbotsford
8                            NaN                              Docklands
9                            NaN                         East Melbourne
10                           NaN                          East St Kilda
11                           NaN                                 Elwood
12                           Na